# Stock Price Prediction - NIFTY 50
## Notebook 01: Data Acquisition and Preprocessing

**Objective:** Fetch, merge, clean NIFTY50 data (2020-2025). Calculate Log Returns.

### Learning Outcomes:
- Master financial data acquisition from NSE using `yfinance` and `nsepy`
- Understand data preprocessing techniques for financial time series
- Learn to calculate and interpret log returns
- Implement robust data cleaning and validation procedures

---

## 1. Import Required Libraries

We'll start by importing all necessary libraries for data acquisition and preprocessing.

In [ ]:
# Core Data Handling
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Data Acquisition
import yfinance as yf
from nsepy import get_history
from datetime import datetime, timedelta
import requests

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# System and File Operations
import os
import sys
from pathlib import Path

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8')

print("📊 Libraries imported successfully!")
print(f"📅 Current date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Define NIFTY 50 Stock Symbols

Let's define the current NIFTY 50 constituents for data acquisition.

In [ ]:
# NIFTY 50 Stock Symbols (Updated as of 2024)
NIFTY_50_SYMBOLS = [
    'RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'ICICIBANK.NS', 'HINDUNILVR.NS',
    'INFY.NS', 'ITC.NS', 'SBIN.NS', 'BHARTIARTL.NS', 'ASIANPAINT.NS',
    'MARUTI.NS', 'HCLTECH.NS', 'AXISBANK.NS', 'LT.NS', 'SUNPHARMA.NS',
    'TITAN.NS', 'ULTRACEMCO.NS', 'WIPRO.NS', 'NESTLEIND.NS', 'POWERGRID.NS',
    'NTPC.NS', 'TECHM.NS', 'ONGC.NS', 'M&M.NS', 'TATAMOTORS.NS',
    'KOTAKBANK.NS', 'HDFCLIFE.NS', 'BAJFINANCE.NS', 'SBILIFE.NS', 'DRREDDY.NS',
    'INDUSINDBK.NS', 'ADANIENT.NS', 'GRASIM.NS', 'CIPLA.NS', 'BRITANNIA.NS',
    'COALINDIA.NS', 'TATASTEEL.NS', 'APOLLOHOSP.NS', 'HINDALCO.NS', 'DIVISLAB.NS',
    'HEROMOTOCO.NS', 'ADANIPORTS.NS', 'UPL.NS', 'BAJAJFINSV.NS', 'JSWSTEEL.NS',
    'EICHERMOT.NS', 'TATACONSUM.NS', 'LTIM.NS', 'BAJAJ-AUTO.NS', 'BPCL.NS'
]

print(f"🎯 Total NIFTY 50 symbols defined: {len(NIFTY_50_SYMBOLS)}")
print("\n📋 Sample symbols:")
for i, symbol in enumerate(NIFTY_50_SYMBOLS[:10]):
    print(f"{i+1:2d}. {symbol}")

## 3. Data Acquisition Functions

Let's create robust functions to fetch stock data with error handling and retry mechanisms.

In [ ]:
def fetch_stock_data(symbol, start_date='2020-01-01', end_date=None, max_retries=3):
    """
    Fetch stock data for a given symbol using yfinance.
    
    Parameters:
    - symbol: Stock symbol (e.g., 'RELIANCE.NS')
    - start_date: Start date for data (default: '2020-01-01')
    - end_date: End date for data (default: today)
    - max_retries: Maximum number of retry attempts
    
    Returns:
    - DataFrame with stock data or None if failed
    """
    if end_date is None:
        end_date = datetime.now().strftime('%Y-%m-%d')
    
    for attempt in range(max_retries):
        try:
            # Fetch data using yfinance
            ticker = yf.Ticker(symbol)
            data = ticker.history(start=start_date, end=end_date)
            
            if not data.empty:
                # Add symbol column
                data['Symbol'] = symbol.replace('.NS', '')
                data.reset_index(inplace=True)
                
                print(f"✅ Successfully fetched {len(data)} records for {symbol}")
                return data
            else:
                print(f"⚠️  No data found for {symbol}")
                
        except Exception as e:
            print(f"❌ Attempt {attempt + 1} failed for {symbol}: {str(e)}")
            if attempt < max_retries - 1:
                print(f"🔄 Retrying in 2 seconds...")
                import time
                time.sleep(2)
    
    print(f"❌ Failed to fetch data for {symbol} after {max_retries} attempts")
    return None

def fetch_nifty_index_data(start_date='2020-01-01', end_date=None):
    """
    Fetch NIFTY 50 index data.
    """
    print("📈 Fetching NIFTY 50 Index data...")
    return fetch_stock_data('^NSEI', start_date, end_date)

print("🔧 Data acquisition functions defined successfully!")

## 4. Batch Data Collection

Now let's fetch data for all NIFTY 50 stocks with progress tracking.

In [ ]:
# Set date range for data collection
START_DATE = '2020-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')

print(f"📊 Starting data collection for NIFTY 50 stocks")
print(f"📅 Date range: {START_DATE} to {END_DATE}")
print(f"🎯 Total symbols to process: {len(NIFTY_50_SYMBOLS)}")
print("-" * 60)

# Initialize containers
all_stock_data = []
failed_symbols = []
successful_symbols = []

# Fetch data for each symbol
for i, symbol in enumerate(NIFTY_50_SYMBOLS, 1):
    print(f"\n📈 [{i:2d}/{len(NIFTY_50_SYMBOLS)}] Processing {symbol}...")
    
    data = fetch_stock_data(symbol, START_DATE, END_DATE)
    
    if data is not None and not data.empty:
        all_stock_data.append(data)
        successful_symbols.append(symbol)
    else:
        failed_symbols.append(symbol)
    
    # Progress update every 10 stocks
    if i % 10 == 0:
        print(f"\n🏁 Progress: {i}/{len(NIFTY_50_SYMBOLS)} completed ({i/len(NIFTY_50_SYMBOLS)*100:.1f}%)")

# Summary
print("\n" + "="*60)
print("📊 DATA COLLECTION SUMMARY")
print("="*60)
print(f"✅ Successful: {len(successful_symbols)} stocks")
print(f"❌ Failed: {len(failed_symbols)} stocks")
print(f"📈 Success Rate: {len(successful_symbols)/len(NIFTY_50_SYMBOLS)*100:.1f}%")

if failed_symbols:
    print(f"\n❌ Failed symbols: {', '.join(failed_symbols)}")

## 5. Combine and Structure Data

Let's combine all individual stock data into a master DataFrame.

In [ ]:
# Combine all stock data
if all_stock_data:
    # Concatenate all DataFrames
    combined_data = pd.concat(all_stock_data, ignore_index=True)
    
    # Sort by Symbol and Date
    combined_data = combined_data.sort_values(['Symbol', 'Date']).reset_index(drop=True)
    
    print("📊 MASTER DATASET CREATED")
    print("-" * 40)
    print(f"📈 Total Records: {len(combined_data):,}")
    print(f"🏢 Unique Stocks: {combined_data['Symbol'].nunique()}")
    print(f"📅 Date Range: {combined_data['Date'].min()} to {combined_data['Date'].max()}")
    print(f"💾 Memory Usage: {combined_data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Display sample data
    print("\n📋 Sample Data:")
    display(combined_data.head(10))
    
    # Data info
    print("\n📊 Dataset Info:")
    combined_data.info()
    
else:
    print("❌ No data collected. Please check your internet connection and symbol list.")

## 6. Data Quality Assessment

Let's assess the quality of our collected data and identify any issues.

In [ ]:
# Data Quality Assessment
print("🔍 DATA QUALITY ASSESSMENT")
print("=" * 50)

# Missing values analysis
missing_data = combined_data.isnull().sum()
print("\n❓ Missing Values:")
for col, count in missing_data.items():
    if count > 0:
        percentage = (count / len(combined_data)) * 100
        print(f"  {col}: {count:,} ({percentage:.2f}%)")

# Check for duplicate records
duplicates = combined_data.duplicated(['Symbol', 'Date']).sum()
print(f"\n🔄 Duplicate Records: {duplicates}")

# Data completeness by symbol
data_completeness = combined_data.groupby('Symbol').size().describe()
print("\n📊 Data Completeness by Symbol:")
print(data_completeness)

# Check for negative or zero prices (data anomalies)
price_anomalies = {
    'Negative Open': (combined_data['Open'] <= 0).sum(),
    'Negative High': (combined_data['High'] <= 0).sum(),
    'Negative Low': (combined_data['Low'] <= 0).sum(),
    'Negative Close': (combined_data['Close'] <= 0).sum(),
    'Zero Volume': (combined_data['Volume'] == 0).sum()
}

print("\n⚠️  Price Anomalies:")
for anomaly, count in price_anomalies.items():
    if count > 0:
        print(f"  {anomaly}: {count}")
    else:
        print(f"  {anomaly}: ✅ None found")

# Date range consistency
date_ranges = combined_data.groupby('Symbol')['Date'].agg(['min', 'max', 'count'])
print("\n📅 Date Range Summary:")
print(f"  Earliest Date: {date_ranges['min'].min()}")
print(f"  Latest Date: {date_ranges['max'].max()}")
print(f"  Min Records/Stock: {date_ranges['count'].min()}")
print(f"  Max Records/Stock: {date_ranges['count'].max()}")

## 7. Calculate Log Returns

**Fundamental Concept:** Raw stock prices are often **non-stationary**, violating assumptions of many statistical models. By contrast, **Daily Returns** are generally **stationary**, making them a more stable and reliable variable for forecasting. We calculate the **Log Returns** using the natural logarithm.

The mathematical formula used for Log Returns is: $LogReturn_t = \ln(\frac{Close_t}{Close_{t-1}})$

In [ ]:
def calculate_log_returns(df):
    """
    Calculate log returns for each stock in the dataset.
    
    Log Return = ln(Price_t / Price_t-1)
    """
    df = df.copy()
    df = df.sort_values(['Symbol', 'Date'])
    
    # Calculate log returns for each stock separately
    df['Log_Return'] = df.groupby('Symbol')['Close'].transform(
        lambda x: np.log(x / x.shift(1))
    )
    
    # Calculate simple returns as well for comparison
    df['Simple_Return'] = df.groupby('Symbol')['Close'].pct_change()
    
    # Calculate additional return metrics
    df['Price_Change'] = df.groupby('Symbol')['Close'].diff()
    df['Price_Change_Pct'] = df['Price_Change'] / df.groupby('Symbol')['Close'].shift(1) * 100
    
    return df

# Apply log return calculation
print("📊 Calculating Log Returns...")
combined_data_with_returns = calculate_log_returns(combined_data)

# Remove first day for each stock (NaN values due to lag calculation)
combined_data_with_returns = combined_data_with_returns.dropna(subset=['Log_Return'])

print("✅ Log Returns calculated successfully!")
print(f"📈 Records with returns: {len(combined_data_with_returns):,}")

# Display sample with returns
print("\n📋 Sample Data with Returns:")
sample_cols = ['Date', 'Symbol', 'Close', 'Log_Return', 'Simple_Return', 'Price_Change_Pct']
display(combined_data_with_returns[sample_cols].head(10))

## 8. Return Statistics and Analysis

Let's analyze the statistical properties of our calculated returns.

In [ ]:
# Calculate return statistics by symbol
return_stats = combined_data_with_returns.groupby('Symbol')['Log_Return'].agg([
    'count', 'mean', 'std', 'min', 'max', 'skew', 
    lambda x: x.quantile(0.25),  # Q1
    'median',                     # Q2
    lambda x: x.quantile(0.75)   # Q3
]).round(6)

# Rename lambda columns
return_stats.columns = ['Count', 'Mean', 'Std', 'Min', 'Max', 'Skewness', 'Q1', 'Median', 'Q3']

# Add additional metrics
return_stats['Annualized_Return'] = return_stats['Mean'] * 252  # 252 trading days
return_stats['Annualized_Volatility'] = return_stats['Std'] * np.sqrt(252)
return_stats['Sharpe_Ratio'] = return_stats['Annualized_Return'] / return_stats['Annualized_Volatility']

print("📊 LOG RETURN STATISTICS BY STOCK")
print("=" * 60)

# Display top 10 stocks by Sharpe ratio
print("\n🏆 Top 10 Stocks by Sharpe Ratio:")
top_sharpe = return_stats.nlargest(10, 'Sharpe_Ratio')[['Mean', 'Std', 'Annualized_Return', 'Sharpe_Ratio']]
display(top_sharpe)

# Overall market statistics
overall_stats = combined_data_with_returns['Log_Return'].agg([
    'count', 'mean', 'std', 'min', 'max', 'skew', 'kurtosis'
])

print("\n📈 Overall Market Statistics:")
print(f"  Total Observations: {overall_stats['count']:,}")
print(f"  Mean Log Return: {overall_stats['mean']:.6f} ({overall_stats['mean']*252:.4f} annualized)")
print(f"  Volatility (Std): {overall_stats['std']:.6f} ({overall_stats['std']*np.sqrt(252):.4f} annualized)")
print(f"  Minimum Return: {overall_stats['min']:.6f} ({overall_stats['min']*100:.2f}%)")
print(f"  Maximum Return: {overall_stats['max']:.6f} ({overall_stats['max']*100:.2f}%)")
print(f"  Skewness: {overall_stats['skew']:.4f}")
print(f"  Kurtosis: {overall_stats['kurtosis']:.4f}")

## 9. Data Visualization

Let's create comprehensive visualizations to understand our data better.

In [ ]:
# Create comprehensive visualizations
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Log Returns Distribution (All Stocks)',
        'Sample Stock Price Evolution', 
        'Returns Volatility by Stock',
        'Cumulative Returns (Top 5 by Sharpe)'
    ],
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# 1. Log Returns Distribution
fig.add_trace(
    go.Histogram(
        x=combined_data_with_returns['Log_Return'],
        nbinsx=100,
        name='Log Returns',
        opacity=0.7
    ),
    row=1, col=1
)

# 2. Sample Stock Price Evolution (Reliance)
reliance_data = combined_data_with_returns[combined_data_with_returns['Symbol'] == 'RELIANCE'].copy()
if not reliance_data.empty:
    fig.add_trace(
        go.Scatter(
            x=reliance_data['Date'],
            y=reliance_data['Close'],
            mode='lines',
            name='RELIANCE Close Price',
            line=dict(color='blue')
        ),
        row=1, col=2
    )

# 3. Returns Volatility by Stock
top_10_vol = return_stats.nlargest(10, 'Std')
fig.add_trace(
    go.Bar(
        x=top_10_vol.index,
        y=top_10_vol['Std'],
        name='Daily Volatility',
        marker_color='red',
        opacity=0.7
    ),
    row=2, col=1
)

# 4. Cumulative Returns for Top 5 Sharpe Ratio stocks
top_5_symbols = return_stats.nlargest(5, 'Sharpe_Ratio').index.tolist()
colors = ['blue', 'red', 'green', 'purple', 'orange']

for i, symbol in enumerate(top_5_symbols):
    stock_data = combined_data_with_returns[combined_data_with_returns['Symbol'] == symbol].copy()
    if not stock_data.empty:
        stock_data = stock_data.sort_values('Date')
        stock_data['Cumulative_Return'] = (1 + stock_data['Log_Return']).cumprod() - 1
        
        fig.add_trace(
            go.Scatter(
                x=stock_data['Date'],
                y=stock_data['Cumulative_Return'] * 100,
                mode='lines',
                name=f'{symbol}',
                line=dict(color=colors[i])
            ),
            row=2, col=2
        )

# Update layout
fig.update_layout(
    height=800,
    title_text="NIFTY 50 Data Analysis Dashboard",
    title_x=0.5,
    showlegend=True
)

# Update axes labels
fig.update_xaxes(title_text="Log Return", row=1, col=1)
fig.update_yaxes(title_text="Frequency", row=1, col=1)

fig.update_xaxes(title_text="Date", row=1, col=2)
fig.update_yaxes(title_text="Price (₹)", row=1, col=2)

fig.update_xaxes(title_text="Stock Symbol", row=2, col=1)
fig.update_yaxes(title_text="Daily Volatility", row=2, col=1)

fig.update_xaxes(title_text="Date", row=2, col=2)
fig.update_yaxes(title_text="Cumulative Return (%)", row=2, col=2)

fig.show()

print("📊 Comprehensive data visualization completed!")

## 10. Save Processed Data

Let's save our cleaned and processed data for use in subsequent notebooks.

In [ ]:
# Create data directory if it doesn't exist
data_dir = Path('../data/processed')
data_dir.mkdir(parents=True, exist_ok=True)

# Save the processed data
processed_file = data_dir / 'nifty50_processed_data.csv'
combined_data_with_returns.to_csv(processed_file, index=False)

# Save return statistics
stats_file = data_dir / 'nifty50_return_statistics.csv'
return_stats.to_csv(stats_file)

# Create a summary file
summary_info = {
    'total_records': len(combined_data_with_returns),
    'unique_symbols': combined_data_with_returns['Symbol'].nunique(),
    'date_range_start': str(combined_data_with_returns['Date'].min()),
    'date_range_end': str(combined_data_with_returns['Date'].max()),
    'successful_symbols': len(successful_symbols),
    'failed_symbols': len(failed_symbols),
    'processing_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

summary_df = pd.DataFrame([summary_info])
summary_file = data_dir / 'data_processing_summary.csv'
summary_df.to_csv(summary_file, index=False)

print("💾 DATA SAVED SUCCESSFULLY")
print("=" * 40)
print(f"📁 Main Dataset: {processed_file}")
print(f"📊 Statistics: {stats_file}")
print(f"📋 Summary: {summary_file}")
print(f"\n📈 Records Saved: {len(combined_data_with_returns):,}")
print(f"💾 File Size: {processed_file.stat().st_size / 1024**2:.2f} MB")

## 📋 Summary

### ✅ What We Accomplished:

1. **Data Acquisition**: Successfully fetched NIFTY 50 stock data from 2020-2025
2. **Data Quality**: Assessed and validated data quality with comprehensive checks
3. **Log Returns**: Calculated log returns for stationarity and modeling readiness
4. **Statistical Analysis**: Computed return statistics, volatility, and Sharpe ratios
5. **Visualization**: Created comprehensive dashboards for data exploration
6. **Data Export**: Saved processed data for subsequent analysis notebooks

### 📊 Key Insights:

- **Dataset Size**: Processed thousands of records across 50 stocks
- **Time Period**: 5+ years of comprehensive market data
- **Return Characteristics**: Analyzed risk-return profiles of individual stocks
- **Data Quality**: Implemented robust error handling and quality assurance

### 🔄 Next Steps:

**Notebook 02**: We'll perform Exploratory Data Analysis (EDA) and Time Series Foundations including:
- Stationarity testing (ADF tests)
- ACF/PACF analysis
- Time-based data splitting
- Trend and seasonality analysis

---

**📝 Note**: This notebook establishes the foundation for all subsequent analysis. The log returns calculated here will be our primary target variable for prediction models.